# Choose city

We have chosen four criteria for our company

- Designers like to go to design talks and share knowledge. There must be some nearby companies that also do design.

- 30% of the company staff have at least 1 child.

- Developers like to be near successful tech startups that have raised at least 1 Million dollars.

- Executives like Starbucks A LOT. Ensure there's a starbucks not too far.

In [31]:
from pymongo import MongoClient
import json
import pandas as pd
import geopandas as gpd
import requests
from dotenv import load_dotenv
import os
from functools import reduce
import operator
import folium
from folium import Choropleth, Circle, Marker, Icon, Map
from folium.plugins import HeatMap, MarkerCluster
from src.utils import *
#import libraries

In [2]:
client = MongoClient()#conect to Mongo

In [3]:
client

MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True)

In [4]:
companies = client.Dataironhack.LabMongo
offices = client.Dataironhack.offices
#create collections

In [5]:
comp_info = list(companies.find(
    {"$or":
     [{"$and":
       [{"tag_list":{"$regex":"design"}},
        {"deadpooled_year":{"$eq":None}}]},
      {"$and":
       [{"category_code":"software"},
        {"total_money_raised":{"$regex":"[\$]([1-9])[\.\d]?[M]"}},
        {"deadpooled_year":{"$eq":None}}]}]},
    {"_id":0, "name":1, "offices":1, "category_code":1, "total_money_raised":1}
    
))
#look for companies, on the one hand, that do design tasks, 
#and on the other hand, that are technology companies that have earned more than 1M dollars.
#Also, consider that the companies are still active.

Select from the database the companies that perform design tasks and are still active. In addition to tech companies that have earned more than $1M since their inception, and are still active.
For this, we selected software companies.

In [6]:
offices.insert_many(comp_info)#insert in the collection

In [7]:
office = offices.aggregate([
    {"$unwind":"$offices"}, 
    {"$match": {"offices.city":{"$ne":None}, "offices.latitude":{"$ne":None}, "offices.longitude":{"$ne":None}}},
    {"$set":
     {"geojson":
      {"type":"Point",
       "coordinates":["$offices.longitude","$offices.latitude"]}}},
    {"$project":{"_id":0}}])
#separate offices, eliminate nulls, and create coordinates

In [8]:
offices.drop()#drop the last collection
offices.insert_many(office)#and recreate with new info

In [9]:
offices_info = list(offices.find({},
                            {"_id":0, "name": 1, 
                             "category_code":1, 
                             "offices.city":1, 
                             "offices.latitude":1, 
                             "offices.longitude":1}))


In [10]:
df = pd.DataFrame(offices_info)#create dataframe to check the city that has more companies

In [11]:
df.head()#check

,name,category_code,offices
0,Digg,news,"{'city': 'San Francisco', 'latitude': 37.76472..."
1,Adobe Systems,software,"{'city': 'San Jose', 'latitude': 37.320052, 'l..."
2,Negonation,web,"{'city': '', 'latitude': 40.463667, 'longitude..."
3,SnapTell,software,"{'city': 'Palo Alto', 'latitude': 37.433414, '..."
4,GENWI,mobile,"{'city': 'Los Altos', 'latitude': 33.8171, 'lo..."


In [12]:
df["city"] = df["offices"].apply(extract_city)

In [13]:
df["latitude"] = df["offices"].apply(extract_latitude)

In [14]:
df["longitude"] = df["offices"].apply(extract_longitude)
#separate the dictionary into three columns

In [15]:
df.head()

,name,category_code,offices,city,latitude,longitude
0,Digg,news,"{'city': 'San Francisco', 'latitude': 37.76472...",San Francisco,37.764726,-122.394523
1,Adobe Systems,software,"{'city': 'San Jose', 'latitude': 37.320052, 'l...",San Jose,37.320052,-121.877636
2,Negonation,web,"{'city': '', 'latitude': 40.463667, 'longitude...",,40.463667,-3.749220
3,SnapTell,software,"{'city': 'Palo Alto', 'latitude': 37.433414, '...",Palo Alto,37.433414,-122.128772
4,GENWI,mobile,"{'city': 'Los Altos', 'latitude': 33.8171, 'lo...",Los Altos,33.817100,-111.903500


In [16]:
df = df.drop(columns = ["offices"])#drop the column

In [17]:
df["city"].value_counts()#check the cities

San Francisco           26
New York                24
London                  20
Chicago                 11
Los Angeles             11
                        ..
Kodambakkam, Chennai     1
Greenbelt                1
Oshkosh                  1
Springdale               1
Noida (UP)               1
Name: city, Length: 352, dtype: int64

Create a dataframe of the companies to get their values, and compare which of the cities present has the highest number of selected companies.
As you can see, San Francisco is the city with the most companies, so we selected it for our company.

In [18]:
city_sf = df[df["city"] == "San Francisco"]

In [19]:
office_sanf = offices.aggregate([
    {"$match":
     {"offices.city":"San Francisco"}},
    {"$project":{"_id":0}}])
#choose San Francisco

In [20]:
offices.drop()
offices.insert_many(office_sanf)

In [21]:
offices.create_index([("geojson", "2dsphere")])#create index

'geojson_2dsphere'

In [22]:
map_sf = Map(location=[37.764726, -122.394523],zoom_start=13)#create a map

In [23]:
for i,row in city_sf.iterrows():
    company = {
        "location" : [row["latitude"],row["longitude"]],
        "tooltip" : row["category_code"]}
    if row["category_code"] == "software":
        icon = Icon(color = "blue",
                    prefix = "fa",
                    icon = "laptop",
                    icon_color = "black")
    else:
        icon = Icon(color = "red",
                    prefix = "fa",
                    icon = "briefcase",
                    icon_color = "black")
        
    Marker(**company,icon = icon).add_to(map_sf)
map_sf
#create markers and icons

As can be seen in the map, most of the software companies are together in the same area, while the rest of the companies are more dispersed over a larger area.

# Check the Starbucks

In [24]:
load_dotenv()

True

In [25]:
coord_sf = requests.get(f"https://geocode.xyz/San%20Francisco,US?json=1").json()#take San Francisco coords

In [26]:
coord_sf

{'standard': {'addresst': {},
  'city': 'San Francisco',
  'prov': 'US',
  'countryname': 'United States of America',
  'postal': {},
  'confidence': '0.90'},
 'longt': '-122.40443',
 'alt': {},
 'elevation': {},
 'latt': '37.66881'}

In [27]:
sf_coordinates = get_coordinates(coord_sf)
sf_coordinates#take coordinates

{'type': 'Point', 'coordinates': [-122.40443, 37.66881]}

In [28]:
sanf_coordinates = f"{sf_coordinates.get('coordinates')[1]},{sf_coordinates.get('coordinates')[0]}"

In [29]:
sanf_coordinates

'37.66881,-122.40443'

Get the coordinates of San Francisco to find the Starbucks and schools in the city.

In [32]:
starbucks_list = get_location_from_foursquare_in_SanFrancisco("starbucks")

Calling the Api
Getting the coordinates


In [33]:
starbucks_list[0]

{'name': 'Starbucks', 'latitude': 37.65511627, 'longitude': -122.4109091}

In [34]:
starbucks = client.Dataironhack.starbucks #create collection

In [35]:
starbucks.insert_many(starbucks_list)

In [36]:
star = starbucks.aggregate([
    {"$set": {"geojson" : {"type": "Point", "coordinates" : ["$longitude", "$latitude"]}}},
    {"$match": {"latitude":{"$ne":None}, "longitude":{"$ne":None}}},
    {"$project":{"_id":0}}])
#get geojson

In [37]:
starbucks.drop()
starbucks.insert_many(star)

In [38]:
check_starbucks = list(starbucks.find().limit(1))
check_starbucks
#check

[{'_id': ObjectId('6022559786983bc2e8f75efb'),
  'name': 'Starbucks',
  'latitude': 37.65511627,
  'longitude': -122.4109091,
  'geojson': {'type': 'Point', 'coordinates': [-122.4109091, 37.65511627]}}]

In [39]:
starbucks.create_index([("geojson", "2dsphere")])# get index

'geojson_2dsphere'

In [40]:
map_st = Map(location=[37.764726, -122.394523],zoom_start=12)#create map

In [41]:
st_list = pd.DataFrame(starbucks_list)#create dataframe

In [42]:
st_group = folium.FeatureGroup(name = "Starbucks")

In [43]:
HeatMap(data=st_list[["latitude","longitude"]],radius=16).add_to(st_group)#create heatmap

In [44]:
st_group.add_to(map_st)
map_st

We obtain the coordinates of all the sturbucks in the city, and through the heat map we see that they are spread throughout the city but there is a higher concentration of these establishments near where the companies we have previously selected are located.

# Check the Schools

In [45]:
schools_list = get_location_from_foursquare_in_SanFrancisco_second_mod("school")

Calling the Api
Getting the coordinates


In [46]:
schools_list[0]

{'name': 'All Souls Catholic School',
 'latitude': 37.657142639160156,
 'longitude': -122.4152603149414}

In [47]:
schools = client.Dataironhack.schools

In [48]:
schools.insert_many(schools_list)

In [49]:
sch = schools.aggregate([
    {"$set": {"geojson" : {"type": "Point", "coordinates" : ["$longitude", "$latitude"]}}},
    {"$match": {"latitude":{"$ne":None}, "longitude":{"$ne":None}}},
    {"$project":{"_id":0}}])

In [50]:
schools.drop()
schools.insert_many(sch)

In [51]:
check_school = list(schools.find().limit(1))
check_school

[{'_id': ObjectId('602255a486983bc2e8f75f9c'),
  'name': 'All Souls Catholic School',
  'latitude': 37.657142639160156,
  'longitude': -122.4152603149414,
  'geojson': {'type': 'Point',
   'coordinates': [-122.4152603149414, 37.657142639160156]}}]

In [52]:
schools.create_index([("geojson", "2dsphere")])

'geojson_2dsphere'

In [53]:
sc_list = pd.DataFrame(schools_list)

In [54]:
sc_group = folium.FeatureGroup(name = "School")

In [55]:
HeatMap(data=sc_list[["latitude","longitude"]],radius=16).add_to(sc_group)

In [56]:
sc_group.add_to(map_st)

In [57]:
folium.LayerControl(collapsed=False).add_to(map_st)
map_st

See the map and verify that the concentration of schools is farther away from the business zone than the Starbucks.

# Points

In [58]:
city_sf["category_code"].value_counts()#check the category companies

software       14
advertising     4
cleantech       2
mobile          2
ecommerce       1
design          1
news            1
web             1
Name: category_code, dtype: int64

With all this, let's choose a software company site, but which one?
To facilitate the choice, rate the different proximities to selected establishments 

In [59]:
city_sf_software = city_sf[city_sf["category_code"] == "software"]

In [60]:
software_office = list(offices.find({"category_code":"software"}, {"_id":0}))
software_office.pop(4)
#find the software companies

{'name': 'Telespree',
 'category_code': 'software',
 'total_money_raised': '$1M',
 'offices': {'description': 'Headquarters',
  'address1': '185 Berry Street',
  'address2': '',
  'zip_code': '',
  'city': 'San Francisco',
  'state_code': 'CA',
  'country_code': 'USA',
  'latitude': 37.777092,
  'longitude': -122.391011},
 'geojson': {'type': 'Point', 'coordinates': [-122.391011, 37.777092]}}

In [61]:
designers = client.Dataironhack.designers

In [62]:
design_list=list(offices.find({"category_code":{"$ne":"software"}}))

In [63]:
designers.insert_many(design_list)

In [64]:
design_geo = designers.aggregate([
    {"$set":
     {"geojson":
      {"type":"Point",
       "coordinates":["$offices.longitude","$offices.latitude"]}}},
    {"$project":{"_id":0}}])
#create design collect

In [65]:
designers.drop()
designers.insert_many(design_list)

In [66]:
designers.create_index([("geojson", "2dsphere")])

'geojson_2dsphere'

In [67]:
n_final_list = []
for soft in software_office:
    n_final_dic = {}
    names = soft.get("name")
    i = soft.get("geojson")
    starb = list(starbucks.find({"geojson":{"$near":i, "$maxDistance":500}}))
    design = list(designers.find({"geojson":{"$near":i, "$maxDistance":2000}}))
    software = list(offices.find({"$and":[{"geojson":{"$near":i, "$maxDistance":2000}}, {"category_code": "software"}]}))
    sch = list(schools.find({"geojson":{"$near":i, "$maxDistance":5000}}))
    n_final_dic["name"] = names
    n_final_dic["number_of_starbacks_500m"] = len(starb)
    n_final_dic["number_of_design_companies_2000"] = len(design)
    n_final_dic["number_of_software_companies_2000"] = len(software)
    n_final_dic["number_of_schools_5000"] = len(sch)
    n_final_list.append(n_final_dic)
#get all the near locations

In [68]:
df_software=pd.DataFrame(n_final_list)#create dataframe to points

In [69]:
df_software["points1"]=df_software["number_of_starbacks_500m"].apply(get_points_starbucks)

In [70]:
df_software

,name,number_of_starbacks_500m,number_of_design_companies_2000,number_of_software_companies_2000,number_of_schools_5000,points1
0,Splunk,0,4,11,4,0
1,Globant,1,9,12,5,1
2,Razz,2,6,4,16,1
3,Keibi Technologies,0,9,11,4,0
4,Telespree,1,3,11,6,1
5,Mindjet,0,0,1,0,0
6,Clikthrough,0,4,9,0,0
7,Envivio,0,0,1,22,0
8,Roundbox,2,5,11,4,1
9,Bomgar,3,4,11,3,3


In [71]:
df_software["points2"]=df_software["number_of_design_companies_2000"].apply(get_points_companies)

In [72]:
df_software["points3"]=df_software["number_of_software_companies_2000"].apply(get_points_companies)

In [73]:
df_software["points4"]=df_software["number_of_schools_5000"].apply(get_points_schools)

In [74]:
df_software.head()

,name,number_of_starbacks_500m,number_of_design_companies_2000,number_of_software_companies_2000,number_of_schools_5000,points1,points2,points3,points4
0,Splunk,0,4,11,4,0,1,6,5
1,Globant,1,9,12,5,1,6,6,7
2,Razz,2,6,4,16,1,3,1,7
3,Keibi Technologies,0,9,11,4,0,6,6,5
4,Telespree,1,3,11,6,1,1,6,7


In [75]:
df_software["points"] = df_software["points1"] + df_software["points2"] + df_software["points3"] + df_software["points4"]
#apply the functions and add the points

In [76]:
df_software.drop(columns = ["points1", "points2", "points3", "points4"]).head()

,name,number_of_starbacks_500m,number_of_design_companies_2000,number_of_software_companies_2000,number_of_schools_5000,points
0,Splunk,0,4,11,4,12
1,Globant,1,9,12,5,20
2,Razz,2,6,4,16,12
3,Keibi Technologies,0,9,11,4,17
4,Telespree,1,3,11,6,15


The final scores are in. And the company that meets the most requirements is Globant. So, we already have the point where we are going to install the company. We are going to build our company on top of yours!!!!

# The final Choice

In [77]:
glob = list(offices.find({"name":"Globant"}, {"_id":0, "offices.longitude":1, "offices.latitude":1})) #get Globant

In [78]:
glob

[{'offices': {'latitude': 37.7819286, 'longitude': -122.4041764}}]

In [79]:
for i in glob:
    globant = i.get("offices")

In [80]:
globant
#get coordinates

{'latitude': 37.7819286, 'longitude': -122.4041764}

In [81]:
globant_coords = [i for i in globant.values()]

In [82]:
globant_coords #clean coordinates

[37.7819286, -122.4041764]

In [83]:
map_globant = folium.Map(location = globant_coords, zoom_start=15)#create map

In [84]:
glob_icon = Icon(color = "blue",
             prefix = "fa",
             icon = "id-card",
             icon_color = "black")

glo = Marker(location = globant_coords, icon=glob_icon, tooltip="Globant")
glo.add_to(map_globant)
map_globant

Here we go!